In [1]:
import pandas as pd
import data_util
from tec_an import TecAn
import numpy as np
from data_util import *
from bitstamp import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb


In [2]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = 1234,
                 val_end = 1234,
                 train_start_list = []
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [3]:
win = 40
minus = 15
step =  1

tec = TecAn(windows = win, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(minus, sourceDataGenerator, True, win)

In [4]:
train_x_raw, train_y_raw = online.load_train_data()
train_x, train_y, _closed_prices = get_sequencial_data(train_x_raw, train_y_raw , step)

val_x_raw, val_y_raw = online.load_val_data("ltceur")
val_x, val_y, closed_prices = get_sequencial_data(val_x_raw, val_y_raw , step)


In [5]:
%%time
from catboost import CatBoostClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score


X_train, Y_train = train_x, train_y


clf = CatBoostClassifier()

params = {'iterations': [50, 500, 100],
          'depth': [
                    5,
                    6,
                    10
          ],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10, 15, 20],
          #'eval_metric': ['recall'],
          #'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }


CPU times: user 233 µs, sys: 409 µs, total: 642 µs
Wall time: 1.65 ms


In [6]:
%%time


clf_grid = gridSearch(X_train, Y_train, clf, params, make_scorer(recall_score))

#0.6743010531359825

{'depth': 10, 'iterations': 500, 'l2_leaf_reg': 1e-20, 'leaf_estimation_iterations': 20, 'logging_level': 'Silent', 'loss_function': 'Logloss', 'random_seed': 42}
0.15891683680638788
CPU times: user 3h 25min 31s, sys: 1h 33min 47s, total: 4h 59min 18s
Wall time: 1h 11min 38s


In [7]:
print(clf_grid.best_params_)

{'depth': 10, 'iterations': 500, 'l2_leaf_reg': 1e-20, 'leaf_estimation_iterations': 20, 'logging_level': 'Silent', 'loss_function': 'Logloss', 'random_seed': 42}


In [8]:
#model = CatBoostClassifier(**clf_grid.best_params_)

#model.fit(X_train, Y_train)
#model.fit(x_norm, Y_train)

model = clf_grid.best_estimator_

eval_data(model, X_train, Y_train)

#5473554735547356

              precision    recall  f1-score   support

     class 0       0.90      1.00      0.95     16198
     class 1       1.00      0.78      0.87      7778

    accuracy                           0.93     23976
   macro avg       0.95      0.89      0.91     23976
weighted avg       0.93      0.93      0.92     23976



In [10]:

eval_data(model, val_x, val_y)


              precision    recall  f1-score   support

     class 0       0.62      0.82      0.71      1763
     class 1       0.43      0.21      0.28      1120

    accuracy                           0.59      2883
   macro avg       0.53      0.52      0.50      2883
weighted avg       0.55      0.59      0.54      2883



In [ ]:
from joblib import dump, load
dump(model, 'model/CatBoostClassifier') 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

#model = clf_grid.best_estimator_

clf = Pipeline([
  ('feature_selection', SelectFromModel(
      LinearSVC()
  )
  ),
  ('classification', CatBoostClassifier(logging_level = 'Silent'))
])

clf.fit(X_train, Y_train)



In [ ]:
eval_data(clf, X_train, Y_train)

eval_data(clf, valX, valY)